# Supporting code and data for "..."

In [1]:
%matplotlib inline

import os
import sys
print(f'Python {sys.version}')

import IPython
from IPython.core.display import display, HTML
print(f'IPython {IPython.__version__}')

print('\nLibraries:\n')

import csv
print(f'csv {csv.__version__}')

import matplotlib
import matplotlib.pyplot as plt
print(f'matplotlib {matplotlib.__version__}')

import numpy as np
print(f'numpy {np.__version__}')

import pandas as pd
from pandas.plotting import register_matplotlib_converters
print(f'pandas {pd.__version__}')

import re
print(f're {re.__version__}')

import requests
print(f'requests {requests.__version__}')

#import scipy
#import scipy.stats
#print(f'scipy {scipy.__version__}')


#import statsmodels
#import statsmodels.formula.api as smf
#from statsmodels.stats.outliers_influence import summary_table
#print(f'statsmodels {statsmodels.__version__}')

Python 3.9.6 (default, Jun 28 2021, 08:57:49) 
[GCC 10.3.0]
IPython 7.24.1

Libraries:

csv 1.0
matplotlib 3.4.2
numpy 1.20.3
pandas 1.2.4
re 2.2.1
requests 2.25.1
statsmodels 0.12.2


## Data collection

We use the GitHub GraphQL API because it allows fetching only the information we need, and at a much faster rate (we can get up to 100 nodes in a single request). Getting all the objects of a certain type requires then to repeat the request to go through all the pages of results.

You need to provide a personal `api_token` if you want to get fresh data from GitHub. Otherwise, this notebook will skip the data collection step and load the CSV files from the local filesystem.

In [2]:
api_token = ''

In [3]:
def requestAllPages(query,rows_and_next_variables,filename,columns):
  if api_token == '':
    return
  headers = {'Authorization': f'token {api_token}'}
  url = 'https://api.github.com/graphql'
  rows, variables = rows_and_next_variables(None)
  while len(variables)>0:
    json = {'query':query,'variables':variables.pop()}
    r = requests.post(url=url, json=json, headers=headers)
    if r.status_code == 403:
      print('Unauthorized request:')
      print(json)
    r.raise_for_status() # Abort if unsuccessful request
    new_rows, next_variables = rows_and_next_variables(r.json()['data'])
    rows += new_rows
    variables += next_variables
  if len(rows) > 0:
    with open(filename, 'w') as f:
      writer = csv.writer(f)
      writer.writerow(columns)
      writer.writerows(rows)

We search all PRs where CI minimization was proposed (excluding those authored by Jason Gross, which were mostly to debug the minimizer) and we retrieve all the comments from coqbot-app to know what happened. We only keep the first 15 lines of each comment, to reduce the size of the CSV file, because these lines will contain all the information we need.

Make sure to uncomment the last line and to provide an `api_token` to re-run this.

In [4]:
def fetch_pr_comments():

  query = """
    query commentQuery($number: Int!, $single: Boolean!, $prCursor: String, $commentCursor: String) {
      search(query: "repo:coq/coq coqbot ci minimize", type:ISSUE, first: 10, after: $prCursor) @skip (if: $single) {
        pageInfo {
          endCursor
          hasNextPage
        }
        nodes {
          ... pullRequest
        }
      }
      repository(owner: "coq", name: "coq") @include (if: $single) {
        pullRequest(number: $number) {
          ... pullRequest
        }
      }
    }

    fragment pullRequest on PullRequest {
      number
      author { login }
      comments(first: 50, after: $commentCursor) {
        pageInfo {
          endCursor
          hasNextPage
        }
        nodes {
          createdAt
          author { login }
          bodyText
          databaseId
        }
      }
    }
  """

  def treat_pr(pr):
    rows, variables = [], []
    number = pr['number']
    if pr['author']['login'] != 'JasonGross':
      for comment in pr['comments']['nodes']:
        if comment['author']['login'] == "coqbot-app":
          date = pd.to_datetime(comment['createdAt']).tz_localize(None)
          body = '\\n'.join(comment['bodyText'].split('\n')[:15])
          rows.append([comment['databaseId'],number,date,body])
    if pr['comments']['pageInfo']['hasNextPage']:
      variables += [{
          'single':True,
          'number':number,
          'commentCursor':pr['comments']['pageInfo']['endCursor']
      }]
    return rows, variables

  def rows_and_next_variables(data):
    if data is None:
      return [], [{'single':False,'number':0}]
    else:
      if 'search' in data:
        prs = data['search']
        rows, variables = [], []
        for pr in prs['nodes']:
          if 'number' in pr:
            new_rows, new_variables = treat_pr(pr)
            rows += new_rows
            variables += new_variables
        if prs['pageInfo']['hasNextPage']:
          variables += [{
              'single':False,
              'number':0,
              'prCursor':prs['pageInfo']['endCursor']
          }]
        return rows, variables
      else:
        return treat_pr(data['repository']['pullRequest'])

  requestAllPages(
      query,
      rows_and_next_variables,
      'pr_comments.csv',
      ['id','number','date','body']
  )

# fetch_pr_comments()

## Data processing

We retrieve the saved dataset from disk.

In [5]:
coqbot_comments = pd.read_csv('pr_comments.csv',parse_dates=['date'],index_col=0)

In [6]:
ci_minimize_results = coqbot_comments[coqbot_comments['body'].str.contains('Minimized File')].sort_values('number')
ci_minimize_results

,number,date,body
id,,,
986083609,11966,2021-12-04 19:51:01,Minimized File /github/workspace/builds/coq/co...
985996564,11966,2021-12-04 09:16:38,Minimized File /github/workspace/builds/coq/co...
985961859,11966,2021-12-04 03:59:43,Minimized File /github/workspace/builds/coq/co...
985891061,11966,2021-12-03 22:42:41,Minimized File /github/workspace/builds/coq/co...
886395867,11966,2021-07-26 05:49:12,Minimized File /github/workspace/builds/coq/co...
...,...,...,...
1017955960,15518,2022-01-20 21:47:05,Minimized File /github/workspace/builds/coq/co...
1017957113,15518,2022-01-20 21:48:56,Minimized File /github/workspace/builds/coq/co...
1017959688,15518,2022-01-20 21:53:07,Minimized File /github/workspace/builds/coq/co...


We can see that our dataset includes 50 pull requests for which a reduced test case was produced by the bug minimizer.

In [7]:
minimized_prs = ci_minimize_results['number'].drop_duplicates()
len(minimized_prs)

50

### Matching CI minimize comments

We look for comments marking the beginning and the end of the minimization. We only keep the first run for each pull request and minimized project to avoid double counting minimization examples.

In [8]:
minimization_started_comments = coqbot_comments[coqbot_comments['body'].str.match(r'I (?:have initiated|am now running) minimization at commit [a-z0-9]* (?:for the suggested targets? | on)?')]
minimization_started_comments = minimization_started_comments[~minimization_started_comments.index.duplicated(keep='first')]
targets = minimization_started_comments['body'].str.extractall(r'(?P<target>ci-[^,.\s]*)')
minimization_started_comments = targets.join(minimization_started_comments).sort_values('date').drop_duplicates(subset=['target','number']).set_index(['target','number'])[['date']]
minimization_started_comments

,,date
target,number,
ci-mathcomp,13969,2021-05-21 22:34:08
ci-fourcolor,13969,2021-05-26 14:10:29
ci-equations,13969,2021-05-26 14:10:29
ci-iris,13969,2021-05-26 14:10:29
ci-perennial,13969,2021-05-26 14:10:29
...,...,...
ci-quickchick,15518,2022-01-20 21:17:11
ci-perennial,15518,2022-01-20 21:17:11
ci-metacoq,15518,2022-01-20 21:17:11


For successful minimization runs (i.e., runs that produced a minimized file, and were not automatically restarted after being interrupted by a timeout), we extract information from the headers of the minimized files, such as the expected `coqc` runtime on this file, the initial number of lines, and the final number of lines, or if any module couldn't be inlined.

In [9]:
minimization_success_comments = coqbot_comments[coqbot_comments['body'].str.startswith('Minimized File') & ~coqbot_comments['body'].str.contains('interrupted by timeout, being automatically continued')]
minimization_success_comments = minimization_success_comments.assign(
    runtime = minimization_success_comments['body'].str.extract(r'Expected coqc runtime on this file: ([0-9\.]+) sec').astype(float),
    initial_size = minimization_success_comments['body'].str.extract(r'from original input, then from ([0-9]+) lines').astype(float),
    final_size = minimization_success_comments['body'].str.extract(r'to ([0-9]+) lines \*\)').astype(float),
    inline_failure = minimization_success_comments['body'].str.contains('could not be inlined'),
    truncated = minimization_success_comments['body'].str.contains('Minimized Coq File \(truncated')
)
minimization_success_comments

,number,date,body,runtime,initial_size,final_size,inline_failure,truncated
id,,,,,,,,
996116369,15274,2021-12-16 19:31:28,Minimized File /github/workspace/builds/coq/co...,0.213,321.0,124.0,False,False
1001010494,15400,2021-12-25 12:16:25,Minimized File /github/workspace/builds/coq/co...,1.018,522.0,81.0,True,False
1017939088,15518,2022-01-20 21:22:16,Minimized File /github/workspace/builds/coq/co...,NaN,NaN,NaN,False,True
1017939694,15518,2022-01-20 21:23:10,Minimized File /github/workspace/builds/coq/co...,0.117,442.0,31.0,False,False
1017939985,15518,2022-01-20 21:23:38,Minimized File /github/workspace/builds/coq/co...,0.134,76.0,25.0,False,False
...,...,...,...,...,...,...,...,...
872424689,13107,2021-07-01 17:27:36,Minimized File /github/workspace/builds/coq/co...,NaN,3477.0,121.0,False,False
872633626,13107,2021-07-02 00:39:37,Minimized File /github/workspace/builds/coq/co...,NaN,1465.0,73.0,False,False
872648442,13107,2021-07-02 01:28:51,Minimized File /github/workspace/builds/coq/co...,NaN,2300.0,128.0,False,False


There are a few files for which we couldn't retrieve any info from the headers (or with a header with absurd data). These actually correspond to files where the minimizer was not able to reproduce any error message and thus did not really produce a reduced case. Thus, we should count this together with the minimization failures:

In [ ]:
minimization_false_successes = minimization_success_comments[minimization_success_comments['initial_size'].isna() | (minimization_success_comments['initial_size'] == 0)]
minimization_success_comments = minimization_success_comments[~minimization_success_comments['initial_size'].isna() & (minimization_success_comments['initial_size'] > 0)]
minimization_false_successes

Only 6 reduced cases were recorded with an inline failure:

In [ ]:
minimization_success_comments[minimization_success_comments['inline_failure']==True]

For these reduced test cases, we also compute the total number of removed lines by aggregating the information found in the headers about the number of lines removed at each step.

In [ ]:
line_reduction = minimization_success_comments['body'].str.extractall(r'then from (?P<reduced_from>[0-9]+) lines to (?P<reduced_to>[0-9]+) lines')
line_reduction = line_reduction.assign(total_line_reduction = line_reduction['reduced_from'].astype(int) - line_reduction['reduced_to'].astype(int))
line_reduction = line_reduction.groupby(level=[0]).sum()
minimization_success_comments = minimization_success_comments.join(line_reduction['total_line_reduction'])
minimization_success_comments

In [ ]:
minimization_failure_comments = coqbot_comments[coqbot_comments['body'].str.startswith('Error: Could not minimize file')].append(minimization_false_successes)
minimization_failure_comments

We aggregate successful and failed end minimization comments and only keep the first run for each project and pull request:

In [ ]:
minimization_finished_comments = minimization_success_comments.assign(success=True).append(minimization_failure_comments.assign(success=False))
targets = minimization_finished_comments['body'].str.extract(r'(?P<target>ci-[^)]*)')
minimization_finished_comments = targets.join(minimization_finished_comments).sort_values('date').drop_duplicates(subset=['target','number']).set_index(['target','number'])#[['date','success','runtime']]
minimization_finished_comments

We match these with the comments corresponding to the beginning of the minimization to compute the minimization duration:

In [ ]:
minimization_pairs = minimization_started_comments.join(minimization_finished_comments,lsuffix='_start',rsuffix='_end')
minimization_pairs = minimization_pairs.assign(duration=(minimization_pairs['date_end'] - minimization_pairs['date_start']).dt.seconds)
minimization_pairs

For the successful cases, there are other data that we can analyze:

In [16]:
successful_minimization_pairs = minimization_pairs[minimization_pairs['success'] == True][['duration','runtime','initial_size','final_size','inline_failure','truncated','total_line_reduction']]
successful_minimization_pairs

,,duration,runtime,initial_size,final_size,inline_failure,truncated,total_line_reduction
target,number,,,,,,,
ci-perennial,13969,339.0,NaN,1726.0,3.0,False,False,3455.0
ci-quickchick,13969,534.0,NaN,1119.0,66.0,False,False,1074.0
ci-hott,13126,8548.0,NaN,294.0,483.0,False,False,1669.0
ci-fiat_crypto_legacy,13072,380.0,NaN,116.0,63.0,False,False,87.0
ci-quickchick,13072,1399.0,NaN,121.0,150.0,False,False,377.0
...,...,...,...,...,...,...,...,...
ci-unimath,15518,562.0,0.150,3947.0,79.0,False,False,3908.0
ci-rewriter,15518,359.0,0.117,442.0,31.0,False,False,416.0
ci-quickchick,15518,481.0,0.130,164.0,34.0,False,False,135.0


Proportion of the time the minimizer successfully produced a reduced file:

In [17]:
len(successful_minimization_pairs)/len(minimization_pairs)

0.7236180904522613

In [18]:
successful_minimization_pairs['duration'].describe(percentiles=[0.5,0.6,0.7,0.8,0.9])

count      144.000000
mean      6238.326389
std      11694.359851
min        232.000000
50%       1218.500000
60%       2199.800000
70%       4125.700000
80%       8396.400000
90%      19220.400000
max      73072.000000
Name: duration, dtype: float64

In [19]:
minimization_pairs[minimization_pairs['success'] == False]['duration'].describe(percentiles=[0.5,0.6,0.7,0.8,0.9])

count       45.000000
mean      1141.244444
std       3897.370793
min        165.000000
50%        266.000000
60%        299.800000
70%        334.000000
80%        402.400000
90%        690.400000
max      19012.000000
Name: duration, dtype: float64

When are the reduced test cases fully standalone? No inlining failure:

In [20]:
no_inlining_failure = successful_minimization_pairs[(successful_minimization_pairs['inline_failure'] == False)]
no_inlining_failure

,,duration,runtime,initial_size,final_size,inline_failure,truncated,total_line_reduction
target,number,,,,,,,
ci-perennial,13969,339.0,NaN,1726.0,3.0,False,False,3455.0
ci-quickchick,13969,534.0,NaN,1119.0,66.0,False,False,1074.0
ci-hott,13126,8548.0,NaN,294.0,483.0,False,False,1669.0
ci-fiat_crypto_legacy,13072,380.0,NaN,116.0,63.0,False,False,87.0
ci-quickchick,13072,1399.0,NaN,121.0,150.0,False,False,377.0
...,...,...,...,...,...,...,...,...
ci-unimath,15518,562.0,0.150,3947.0,79.0,False,False,3908.0
ci-rewriter,15518,359.0,0.117,442.0,31.0,False,False,416.0
ci-quickchick,15518,481.0,0.130,164.0,34.0,False,False,135.0


In [21]:
len(no_inlining_failure) / len(successful_minimization_pairs)

0.9652777777777778

No inlining failure *and* not truncated:

In [22]:
fully_standalone = no_inlining_failure[no_inlining_failure['truncated'] == False]
fully_standalone

,,duration,runtime,initial_size,final_size,inline_failure,truncated,total_line_reduction
target,number,,,,,,,
ci-perennial,13969,339.0,NaN,1726.0,3.0,False,False,3455.0
ci-quickchick,13969,534.0,NaN,1119.0,66.0,False,False,1074.0
ci-hott,13126,8548.0,NaN,294.0,483.0,False,False,1669.0
ci-fiat_crypto_legacy,13072,380.0,NaN,116.0,63.0,False,False,87.0
ci-quickchick,13072,1399.0,NaN,121.0,150.0,False,False,377.0
...,...,...,...,...,...,...,...,...
ci-unimath,15518,562.0,0.150,3947.0,79.0,False,False,3908.0
ci-rewriter,15518,359.0,0.117,442.0,31.0,False,False,416.0
ci-quickchick,15518,481.0,0.130,164.0,34.0,False,False,135.0


In [23]:
len(fully_standalone) / len(successful_minimization_pairs)

0.8680555555555556

How much smaller the final size is compared to the initial size?

In [24]:
np.mean(successful_minimization_pairs['final_size']/successful_minimization_pairs['initial_size'])

0.6031942878889917

And compared to the initial size, when including all the dependencies that had to be inlined during the process?

In [25]:
np.mean(successful_minimization_pairs['final_size']/(successful_minimization_pairs['initial_size'] + successful_minimization_pairs['total_line_reduction']))

0.2774005795158734

For how many reduced cases do we have the expected runtime? What results do we get?

In [26]:
minimization_success_comments['runtime'].dropna().describe(percentiles=[0.5,0.8,0.9])

count    54.000000
mean      1.346426
std       4.139106
min       0.095000
50%       0.335500
80%       0.824000
90%       1.131600
max      26.506000
Name: runtime, dtype: float64

In [27]:
successful_minimization_pairs.groupby(['target']).agg(['median','count'])

duration       runtime       initial_size        \
                        median count  median count       median count   
target                                                                  
ci-aac_tactics           232.0     1     NaN     0        279.0     1   
ci-argosy               3163.5     4  0.7140     1        540.0     4   
ci-bbv                   619.0     3  0.0950     1       7591.0     3   
ci-bedrock2             3322.0     9  0.1325     4        533.0     9   
ci-bignums              8529.0     1     NaN     0       1747.0     1   
ci-category_theory       634.0     3  0.2365     2        262.0     3   
ci-color                2758.0     5  1.2085     2        714.0     5   
ci-compcert             1197.0     5  0.3390     1        728.0     5   
ci-coqprime             4121.0     1  0.4880     1        464.0     1   
ci-coquelicot            461.5     2  0.6580     1       3775.0     2   
ci-cross_crypto         5953.0     1     NaN     0        934.0     1   
ci-equations             330.5     2     NaN     0        248.5     2   
ci-fcsl_pcm              936.0     1  0.1590     1        613.0     1   
ci-fiat_crypto          9525.0     4  0.1510     1        242.0     4   
ci-fiat_crypto_legacy    791.5    10  0.2250     3        836.5    10   
ci-fiat_parsers          730.5    10  0.3310     1        645.0    10   
ci-flocq                 834.0     1  0.4580     1        703.0     1   
ci-gappa                 902.0     1  0.4840     1        727.0     1   
ci-geocoq              10291.5     2  1.0840     1       1230.0     2   
ci-hott                 1857.0    13  0.4500     3        565.0    13   
ci-iris                 3397.0     8  0.2650     5        461.0     8   
ci-itauto               2936.0     1  0.3460     1      10995.0     1   
ci-math_classes         1076.5     4  0.1690     1        221.0     4   
ci-mathcomp            13729.0     4  1.0570     2       3520.0     4   
ci-menhir               1709.0     2  1.1340     1        454.0     2   
ci-metacoq               370.0     1     NaN     0       1124.0     1   
ci-mtac2                 326.5     2     NaN     0        251.0     2   
ci-perennial            2672.0     8  0.2465     4        601.5     8   
ci-quickchick            507.5     4  0.1300     1        179.5     4   
ci-relation_algebra      707.0     2     NaN     0        223.5     2   
ci-rewriter              910.0     9  0.2495     2       1339.0     9   
ci-sf                    736.0     1     NaN     0       2288.0     1   
ci-stdlib2               373.0     1     NaN     0        138.0     1   
ci-tlc                  4389.0     1     NaN     0        788.0     1   
ci-unimath              1323.5     4  0.1500     1       2435.0     4   
ci-verdi_raft            777.0     5  0.1825     2        618.0     5   
ci-vst                 15140.5     8  0.9890     3        872.0     8   

                      final_size       total_line_reduction        
                          median count               median count  
target                                                             
ci-aac_tactics              27.0     1                256.0     1  
ci-argosy                  369.0     4                467.0     4  
ci-bbv                      33.0     3               7547.0     3  
ci-bedrock2                464.0     9                293.0     9  
ci-bignums                1719.0     1                335.0     1  
ci-category_theory         105.0     3                256.0     3  
ci-color                   366.0     5                745.0     5  
ci-compcert                362.0     5                 94.0     5  
ci-coqprime                474.0     1                110.0     1  
ci-coquelicot               37.0     2               3742.5     2  
ci-cross_crypto            725.0     1                627.0     1  
ci-equations                31.5     2                221.0     2  
ci-fcsl_pcm                 17.0     1                650.0     1  
ci-fiat_crypt

In [28]:
minimization_pairs_reset = minimization_pairs.reset_index()
minimization_pairs_reset[minimization_pairs_reset['target'].isin(['ci-bignums'])]

,target,number,date_start,date_end,body,runtime,initial_size,final_size,inline_failure,truncated,total_line_reduction,success,duration
51,ci-bignums,13107,2021-06-29 14:41:10,2021-06-29 17:03:19,Minimized File /github/workspace/builds/coq/co...,NaN,1747.0,1719.0,False,True,335.0,True,8529.0
